In [2]:
# Install required libraries
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

# Import necessary libraries
from google.colab import auth
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pickle

# Step 1: Authenticate with Google
auth.authenticate_user()

# Step 2: Load credentials from token.pickle
SCOPES = ['https://www.googleapis.com/auth/documents']
creds = None

# Check if token.pickle exists
try:
    with open('token.pickle', 'rb') as token_file:
        creds = pickle.load(token_file)
except FileNotFoundError:
    print("No existing credentials found. Please authenticate.")

# Refresh credentials if expired
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
elif not creds:
    print("You need to authenticate again by running this code.")

# Save credentials for future use
with open('token.pickle', 'wb') as token_file:
    pickle.dump(creds, token_file)

# Step 3: Build the Google Docs API service
try:
    docs_service = build('docs', 'v1', credentials=creds)
    print("Google Docs API service successfully created!")
except Exception as e:
    print(f"Failed to create Google Docs API service: {e}")

# Step 4: Create a new Google Doc
document = docs_service.documents().create().execute()
document_id = document['documentId']
print(f"Created new document with ID: {document_id}")

# Step 5: Prepare the markdown content for conversion
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Step 6: Convert markdown content into Google Docs formatting
# Define a function to handle checkbox conversion
def insert_checkbox(requests, current_index, text, is_checked=False):
    checkbox = "☑️" if is_checked else "⬜"
    requests.append({
        "insertText": {
            "location": {"index": current_index},
            "text": f"{checkbox} {text}\n"
        }
    })
    return current_index + len(f"{checkbox} {text}\n")  # Update current_index after insertion

# Prepare the requests list
requests = []

# Add the main title (Product Team Sync) as Heading 1
current_index = 1
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Product Team Sync - May 15, 2023\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Product Team Sync - May 15, 2023\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_1"},
        "fields": "namedStyleType"
    }
})
current_index += len("Product Team Sync - May 15, 2023\n")

# Insert attendees section (Heading 2)
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Attendees\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Attendees\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_2"},
        "fields": "namedStyleType"
    }
})
current_index += len("Attendees\n")

# Insert attendees list (bullets)
attendees = [
    "Sarah Chen (Product Lead)",
    "Mike Johnson (Engineering)",
    "Anna Smith (Design)",
    "David Park (QA)"
]
for attendee in attendees:
    requests.append({
        "insertText": {
            "location": {"index": current_index},
            "text": f"- {attendee}\n"
        }
    })
    current_index += len(f"- {attendee}\n")

# Insert Agenda section (Heading 2)
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Agenda\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Agenda\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_2"},
        "fields": "namedStyleType"
    }
})
current_index += len("Agenda\n")

# Insert agenda items (Heading 3 and bullet points)
agenda = [
    ("Sprint Review", [
        "Completed Features", [
            "User authentication flow",
            "Dashboard redesign",
            "Performance optimization",
        ],
        "Pending Items", [
            "Mobile responsive fixes",
            "Beta testing feedback integration"
        ]
    ]),
    ("Current Challenges", [
        "Resource constraints in QA team",
        "Third-party API integration delays",
        "User feedback on new UI", [
            "Navigation confusion",
            "Color contrast issues"
        ]
    ]),
    ("Next Sprint Planning", [
        "Priority Features", [
            "Payment gateway integration",
            "User profile enhancement",
            "Analytics dashboard"
        ],
        "Technical Debt", [
            "Code refactoring",
            "Documentation updates"
        ]
    ])
]

for section, items in agenda:
    # Insert section as Heading 3
    requests.append({
        "insertText": {
            "location": {"index": current_index},
            "text": f"{section}\n"
        }
    })
    requests.append({
        "updateParagraphStyle": {
            "range": {"startIndex": current_index, "endIndex": current_index + len(f"{section}\n")},
            "paragraphStyle": {"namedStyleType": "HEADING_3"},
            "fields": "namedStyleType"
        }
    })
    current_index += len(f"{section}\n")

    # Insert bullet points
    for item in items:
        if isinstance(item, str):  # Simple item
            requests.append({
                "insertText": {
                    "location": {"index": current_index},
                    "text": f"- {item}\n"
                }
            })
            current_index += len(f"- {item}\n")
        elif isinstance(item, list):  # Nested items
            for sub_item in item:
                requests.append({
                    "insertText": {
                        "location": {"index": current_index},
                        "text": f"  - {sub_item}\n"
                    }
                })
                current_index += len(f"  - {sub_item}\n")

# Action items with checkboxes
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Action Items\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Action Items\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_2"},
        "fields": "namedStyleType"
    }
})
current_index += len("Action Items\n")

# Insert action items with checkboxes
current_index = insert_checkbox(requests, current_index, "@sarah: Finalize Q3 roadmap by Friday")
current_index = insert_checkbox(requests, current_index, "@mike: Schedule technical review for payment integration")
current_index = insert_checkbox(requests, current_index, "@anna: Share updated design system documentation")
current_index = insert_checkbox(requests, current_index, "@david: Prepare QA resource allocation proposal")

# Next Steps
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Next Steps\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Next Steps\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_2"},
        "fields": "namedStyleType"
    }
})
current_index += len("Next Steps\n")

# Insert next steps as bullets
next_steps = [
    "Schedule individual team reviews",
    "Update sprint board",
    "Share meeting summary with stakeholders"
]
for step in next_steps:
    requests.append({
        "insertText": {
            "location": {"index": current_index},
            "text": f"- {step}\n"
        }
    })
    current_index += len(f"- {step}\n")

# Notes
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "Notes\n"
    }
})
requests.append({
    "updateParagraphStyle": {
        "range": {"startIndex": current_index, "endIndex": current_index + len("Notes\n")},
        "paragraphStyle": {"namedStyleType": "HEADING_2"},
        "fields": "namedStyleType"
    }
})
current_index += len("Notes\n")

# Insert notes
notes = [
    "Next sync scheduled for May 22, 2023",
    "Platform demo for stakeholders on May 25",
    "Remember to update JIRA tickets"
]
for note in notes:
    requests.append({
        "insertText": {
            "location": {"index": current_index},
            "text": f"- {note}\n"
        }
    })
    current_index += len(f"- {note}\n")

# Footer
requests.append({
    "insertText": {
        "location": {"index": current_index},
        "text": "\n---\nMeeting recorded by: Sarah Chen\nDuration: 45 minutes\n"
    }
})

# Execute the batch update request
docs_service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()
print(f"Document updated successfully with ID: {document_id}")


You need to authenticate again by running this code.
Google Docs API service successfully created!
Created new document with ID: 1s7iErtP_KAGSKq6LEG22u74HTb2salFmGHVWpm1329Y
Document updated successfully with ID: 1s7iErtP_KAGSKq6LEG22u74HTb2salFmGHVWpm1329Y
